# CNN 
### convolutional neurol network for 8 class classification 
To have a 15 or a 30 classifier, we modifie the final fully connected layer     
self.fc3 = nn.Linear(512, 8)  : 8 class classifier   
self.fc3 = nn.Linear(512, 15) : 15 class classifier   
self.fc3 = nn.Linear(512, 30) : 30 class classifier   

*And of course the training and testing data*

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import numpy as np

### CNN model

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input phase map channel, 64 output channels, 2x2 square convolution
        self.conv1 = nn.Conv2d(1, 64, kernel_size=2)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=2)
        self.dropout = nn.Dropout(0.5)
        # an affine operation: y = Wx + b
        #(M-3)x(K-3) with M = 6mics and K = 129frequ bins for each time frame TF
        # --> M-3 = 3 and K-3 = 126
        self.fc1 = nn.Linear(64 * 3 * 126, 512)
        self.fc2 = nn.Linear(512, 512)
        #with resolution of 45° so number of classes is I=8
        self.fc3 = nn.Linear(512, 8) 

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.dropout(x)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)  #No need activation since CrossEntropyLoss makes the Softmax
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


model = Net()
print(model)

### Train set loading

In [ ]:
class Train(Dataset):
    
    #Initialize data, download, etc.
    def __init__(self):
        x = np.load('train_data_8class.npy')
        x = np.reshape(x, (x.shape[0],1,x.shape[1],x.shape[2]))
        y = np.load('train_labels_8class.npy')
        y.astype(int)
        self.len = y.shape[0]
        x = torch.from_numpy(x)
        y = torch.from_numpy(y)
        self.x_data = x.double()
        self.y_data = y
        
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len

In [ ]:
train_set = Train()
train_loader = DataLoader(dataset=train_set, batch_size=512,
                         shuffle=True, num_workers=2)

### Test set loading 

In [ ]:
class Test(Dataset):
    
    #Initialize data, download, etc.
    def __init__(self):
        x = np.load('test_data_8class_speech.npy')
        x = np.reshape(x, (x.shape[0],1,x.shape[1],x.shape[2]))
        y = np.load('test_labels_8class.npy')
        y.astype(int)
        self.len = y.shape[0]
        x = torch.from_numpy(x)
        y = torch.from_numpy(y)
        self.x_data = x.double()
        self.y_data = y
        
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len

In [ ]:
test_set = Test()
test_loader = DataLoader(dataset=test_set, batch_size=512,
                         shuffle=True, num_workers=2)

### Hyperparameters for training the model
1 epoch because of time

In [ ]:
batch_size = 512
num_epoch = 1
learning_rate = 0.001

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### Training the model 

In [ ]:
model = model.double()
for epoch in range(num_epoch):
    for i, data in enumerate(train_loader, 0):
        #get the inputs
        inputs, labels = data
        
        #wrap them in variable
        inputs, labels = Variable(inputs), Variable(labels)
        
        #forward pass : compute predicted y by passing x to the model
        y_pred = model(inputs)
        
        #compute and print loss 
        loss = criterion(y_pred, labels)
        
        #print(epoch, i, loss.data[0])
        print(epoch, i, loss.data.item())
        
        #zero gradients, performs a backward pass, and update the weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

### Testing the model 
Calculation of the accuracy of the model on the test set (test phase maps)

In [ ]:
# calculate the accuracy of the trained model on the testing data 

correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print(total)

print('Accuracy of the network on the test phase maps: {:0.2f} %' .format(100 * correct / total))